In [171]:
from typing import List
from dataclasses import fields
import modelspec
from modelspec import field, instance_of, optional
from modelspec.base_types import Base, value_expr_types, ValueExprType, print_v
from typing import List, Dict, Any, Optional
from modelspec.utils import load_xml, load_json, load_yaml
import xml.dom.minidom
from pathlib import Path
import sys
import xml.dom.minidom

In [141]:
@modelspec.define
class Population(Base):
    """
    Some description...

    Args:
        id: The id of the population
        component: the component to use in the population
        size: the size of the population
    """

    id: str = field(validator=instance_of(str))
    component: str = field(default=None, validator=optional(instance_of(str)))
    size: int = field(default=None, validator=optional(instance_of(int)))


@modelspec.define
class ExplicitInput(Base):
    """
    Some description...

    Args:
        target: the target of the input
        input: the input, e.g. pulseGenerator
    """

    target: str = field(default=None, validator=optional(instance_of(str)))
    input: str = field(default=None, validator=optional(instance_of(str)))


@modelspec.define
class Network(Base):
    """
    Some description...

    Args:
        id: The id of the network
        populations: the pops in the net
    """

    id: str = field(validator=instance_of(str))

    populations: List[Population] = field(factory=list)
    explicitInputs: List[ExplicitInput] = field(factory=list)


@modelspec.define
class PulseGenerator(Base):
    """
    Some description...

    Args:
        id: The id of the pulseGenerator
        delay: the delay
        duration: the duration
        amplitude: the amplitude
    """

    id: str = field(validator=instance_of(str))
    delay: str = field(validator=instance_of(str))
    duration: str = field(validator=instance_of(str))
    amplitude: str = field(validator=instance_of(str))


@modelspec.define
class Izhikevich2007Cell(Base):
    """
    Some description...

    Args:
        id: The id of the cell...
    """

    id: str = field(validator=instance_of(str))

    C: str = field(validator=instance_of(str))
    v0: str = field(validator=instance_of(str))
    k: str = field(validator=instance_of(str))
    vr: str = field(validator=instance_of(str))
    vt: str = field(validator=instance_of(str))
    vpeak: str = field(validator=instance_of(str))
    a: str = field(validator=instance_of(str))
    b: str = field(validator=instance_of(str))
    c: str = field(validator=instance_of(str))
    d: str = field(validator=instance_of(str))


@modelspec.define
class NeuroML(Base):
    """
    Some description...

    Args:
        id: The id of the NeuroML 2 document
        version: NeuroML version used
        networks: The networks present
    """

    id: str = field(validator=instance_of(str))
    version: str = field(validator=instance_of(str))

    izhikevich2007Cells: List[Izhikevich2007Cell] = field(factory=list)
    pulseGenerators: List[PulseGenerator] = field(factory=list)
    networks: List[Network] = field(factory=list)


nml_doc = NeuroML(id="TestNeuroML", version="NeuroML_v2.3")

izh = Izhikevich2007Cell(
        id="izh2007RS0",
        C="100pF",
        v0="-60mV",
        k="0.7nS_per_mV",
        vr="-60mV",
        vt="-40mV",
        vpeak="35mV",
        a="0.03per_ms",
        b="-2nS",
        c="-50.0mV",
        d="100pA",
)
nml_doc.izhikevich2007Cells.append(izh)

pg = PulseGenerator(
        id="pulseGen_0", delay="100ms", duration="800ms", amplitude="0.07 nA"
)
nml_doc.pulseGenerators.append(pg)

net = Network(id="IzNet")
nml_doc.networks.append(net)

net.populations.append(Population("IzhPop0", component="izh2007RS0", size=1))
net.explicitInputs.append(ExplicitInput(target="IzhPop0[0]", input="pulseGen_0"))
nml_dict = nml_doc.to_dict()

In [142]:
# Saving xml file
nml_doc.to_xml_file()

'TestNeuroML.xml'

In [143]:
# Saving xml yaml file
nml_doc.to_yaml_file()

'TestNeuroML.yaml'

In [144]:
# Saving json file
nml_doc.to_json_file()

'TestNeuroML.json'

In [145]:
# Loading xml file
loaded_xml = load_xml('TestNeuroML.xml')
loaded_xml

{'TestNeuroML': {'version': 'NeuroML_v2.3',
  'izhikevich2007Cells': {'izh2007RS0': {'C': '100pF',
    'v0': '-60mV',
    'k': '0.7nS_per_mV',
    'vr': '-60mV',
    'vt': '-40mV',
    'vpeak': '35mV',
    'a': '0.03per_ms',
    'b': '-2nS',
    'c': '-50.0mV',
    'd': '100pA'}},
  'pulseGenerators': {'pulseGen_0': {'delay': '100ms',
    'duration': '800ms',
    'amplitude': '0.07 nA'}},
  'networks': {'IzNet': {'populations': {'IzhPop0': {'component': 'izh2007RS0',
      'size': 1}},
    'explicitInputs': [{'target': 'IzhPop0[0]', 'input': 'pulseGen_0'}]}}}}

In [146]:
# Loading yaml file
loaded_yaml = load_yaml('TestNeuroML.yaml')
loaded_yaml

{'TestNeuroML': {'version': 'NeuroML_v2.3',
  'izhikevich2007Cells': {'izh2007RS0': {'C': '100pF',
    'v0': '-60mV',
    'k': '0.7nS_per_mV',
    'vr': '-60mV',
    'vt': '-40mV',
    'vpeak': '35mV',
    'a': '0.03per_ms',
    'b': '-2nS',
    'c': '-50.0mV',
    'd': '100pA'}},
  'pulseGenerators': {'pulseGen_0': {'delay': '100ms',
    'duration': '800ms',
    'amplitude': '0.07 nA'}},
  'networks': {'IzNet': {'populations': {'IzhPop0': {'component': 'izh2007RS0',
      'size': 1}},
    'explicitInputs': [{'target': 'IzhPop0[0]', 'input': 'pulseGen_0'}]}}}}

In [147]:
# Loading json file
loaded_json = load_json('TestNeuroML.json')
loaded_json

{'TestNeuroML': {'version': 'NeuroML_v2.3',
  'izhikevich2007Cells': {'izh2007RS0': {'C': '100pF',
    'v0': '-60mV',
    'k': '0.7nS_per_mV',
    'vr': '-60mV',
    'vt': '-40mV',
    'vpeak': '35mV',
    'a': '0.03per_ms',
    'b': '-2nS',
    'c': '-50.0mV',
    'd': '100pA'}},
  'pulseGenerators': {'pulseGen_0': {'delay': '100ms',
    'duration': '800ms',
    'amplitude': '0.07 nA'}},
  'networks': {'IzNet': {'populations': {'IzhPop0': {'component': 'izh2007RS0',
      'size': 1}},
    'explicitInputs': [{'target': 'IzhPop0[0]', 'input': 'pulseGen_0'}]}}}}

In [148]:
nml_dict = nml_doc.to_dict()

In [149]:
def check(data_1, data_2):
    if data_1 == data_2:
        print("True")
    else:
        print("False")

In [150]:
# Comparing loaded xml with loaded json file
check(loaded_xml, loaded_json)

True


In [151]:
# Comparing loaded xml with loaded original dictionary
check(loaded_xml, nml_dict)

True


In [152]:
# Comparing loaded xml with loaded yaml file
check(loaded_xml, loaded_yaml)

True


# Using the test classes to test the xml serialization

In [197]:
# Some test modelspec classes to use in the tests


@modelspec.define
class NewCell(Base):
    """
    A new cell definition

    Args:
        id: the cell id
        neuroml2_source_file: The path to the source file
    """

    id: str = field(validator=instance_of(str))
    neuroml2_source_file: str = field(
        default=None, validator=optional(instance_of(str))
    )


@modelspec.define
class NewSynapse(Base):
    """
    A new synapse definition

    Args:
        id: the synapse id
        neuroml2_source_file: The path to the source file
        tested: A boolean attribute
    """

    id: str = field(validator=instance_of(str))
    neuroml2_source_file: str = field(
        default=None, validator=optional(instance_of(str))
    )
    tested: bool = field(default=None, validator=optional(instance_of(bool)))


@modelspec.define
class NewRandomConnectivity(Base):
    """
    A new random connectivity definition

    Args:
        probability: Random probability of connection

    """

    probability: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )


@modelspec.define
class NewNetwork(Base):
    """
    A new network definition

    Args:
        id: a unique identifier for the network
        cells: a list of cells
        synapses: a list of synapses
        version: Information on verson of NeuroMLlite
        seed: Seed for random number generator used when building network
        stable: Testing...
        parameters: Dictionary of global parameters for the network
        random_connectivity: Use random connectivity
    """

    id: str = field(validator=instance_of(str))
    cells: List[NewCell] = field(factory=list)
    synapses: List[NewSynapse] = field(factory=list)
    version: str = field(default="NeuroMLlite 0.0", validator=instance_of(str))
    seed: int = field(default=None, validator=optional(instance_of(int)))
    stable: bool = field(default=None, validator=optional(instance_of(bool)))
    parameters: Dict[str, Any] = field(
        default=None, validator=optional(instance_of(dict))
    )
    random_connectivity: NewRandomConnectivity = field(
        default=None, validator=optional(instance_of(NewRandomConnectivity))
    )
    ee0: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )
    ee1: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )
    ee2: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )
    ee3: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )
    ee4: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )
    ee5: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )
    ee6: ValueExprType = field(
        default=None, validator=optional(instance_of(value_expr_types))
    )



In [198]:
tmp_path = '.'
net = NewNetwork(id="netid", parameters={"size": 3, "name": None})

# Some tests on what's allowed
net.ee0 = "str"
net.ee1 = {"a": 2}
net.ee2 = 1
net.ee3 = 1.1
net.ee4 = True
net.ee5 = [1, 2]
net.ee6 = None

cell = NewCell(id="cellid1")
cell.neuroml2_source_file = "nnn"
cell2 = NewCell(id="cellid2")
cell2.neuroml2_source_file = "nnn2"
# net.cells.append(cell)

print(net)
print(net.cells)
print(net)
"""  """
net.cells.append(cell)
net.cells.append(cell2)

syn0 = NewSynapse(id="syn0", neuroml2_source_file=None, tested=True)
net.synapses.append(syn0)
syn1 = NewSynapse(id="syn1", neuroml2_source_file="xx", tested=None)
net.synapses.append(syn1)

rc = NewRandomConnectivity(probability=0.01)
net.random_connectivity = rc
net.stable = False
print(rc)
print(net)


NewNetwork(id='netid', cells=[], synapses=[], version='NeuroMLlite 0.0', seed=None, stable=None, parameters={'size': 3, 'name': None}, random_connectivity=None, ee0='str', ee1={'a': 2}, ee2=1, ee3=1.1, ee4=True, ee5=[1, 2], ee6=None)
[]
NewNetwork(id='netid', cells=[], synapses=[], version='NeuroMLlite 0.0', seed=None, stable=None, parameters={'size': 3, 'name': None}, random_connectivity=None, ee0='str', ee1={'a': 2}, ee2=1, ee3=1.1, ee4=True, ee5=[1, 2], ee6=None)
NewRandomConnectivity(probability=0.01)
NewNetwork(id='netid', cells=[NewCell(id='cellid1', neuroml2_source_file='nnn'), NewCell(id='cellid2', neuroml2_source_file='nnn2')], synapses=[NewSynapse(id='syn0', neuroml2_source_file=None, tested=True), NewSynapse(id='syn1', neuroml2_source_file='xx', tested=None)], version='NeuroMLlite 0.0', seed=None, stable=False, parameters={'size': 3, 'name': None}, random_connectivity=NewRandomConnectivity(probability=0.01), ee0='str', ee1={'a': 2}, ee2=1, ee3=1.1, ee4=True, ee5=[1, 2], ee6=

In [199]:
str_orig = str(net)

filenamej = str(Path(tmp_path) / f"{net.id}.json")
net.to_json_file(filenamej)

filenamey = str(Path(tmp_path) / f"{net.id}.yaml")
# net.id = net.id+'_yaml'
net.to_yaml_file(filenamey)

filenamex = str(Path(tmp_path) / f"{net.id}.xml")
net.to_xml_file(filenamex)

from modelspec.utils import load_json, load_yaml, load_xml

dataj = load_json(filenamej)
print_v("Loaded network specification from %s" % filenamej)
netj = NewNetwork.from_dict(dataj)

str_netj = str(netj)

datay = load_yaml(filenamey)
print_v("Loaded network specification from %s" % filenamey)

nety = NewNetwork.from_dict(datay)
str_nety = str(nety)

datax = load_xml(filenamex)
print_v("Loaded network specification from %s" % filenamex)

netx = NewNetwork.from_dict(datax)
str_netx = str(netx)

print("----- Before -----")
print(str_orig)
print("----- After via %s -----" % filenamej)
print(str_netj)
print("----- After via %s -----" % filenamey)
print(str_nety)
print("----- After via %s -----" % filenamex)
print(str_netx)

print("Test JSON..")
if sys.version_info[0] == 2:
    assert len(str_orig) == len(
        str_netj
    )  # Order not preserved in py2, just test len
else:
    assert str_orig == str_netj

print("Test YAML..")
if sys.version_info[0] == 2:
    assert len(str_orig) == len(
        str_nety
    )  # Order not preserved in py2, just test len
else:
    assert str_orig == str_nety

#     print("Test XML..")
#     if sys.version_info[0] == 2:
#         assert len(str_orig) == len(
#             str_netx
#         )  # Order not preserved in py2, just test len
#     else:
#         assert str_orig == str_netx

print("Test EvaluableExpressions")
for i in range(7):
    assert eval("net.ee%i" % i) == eval("netj.ee%i" % i)
    assert eval("net.ee%i" % i) == eval("nety.ee%i" % i)
    #assert eval("net.ee%i" % i) == eval("netx.ee%i" % i)

modelspec >>> Loaded network specification from netid.json
modelspec >>> Loaded network specification from netid.yaml
modelspec >>> Loaded network specification from netid.xml


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [139]:
# Saving to xml file
net.to_xml_file()

'netid.xml'

In [65]:
# Saving to json file
net.to_json_file()

'netid.json'

In [76]:
# Saving to yaml file
net.to_yaml_file()

'netid.yaml'

In [69]:
# Loading xml file
loaded_xml_2 = load_xml('netid.xml')
loaded_xml_2

{'netid': {'version': 'NeuroMLlite 0.0',
  'seed': None,
  'stable': False,
  'parameters': "{'size': 3, 'name': None}",
  'random_connectivity': 'NewRandomConnectivity(probability=0.01)',
  'ee0': 'str',
  'ee1': "{'a': 2}",
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': '[1, 2]',
  'ee6': None,
  'newCells': [{'cellid1': {'neuroml2_source_file': 'nnn'}},
   {'cellid2': {'neuroml2_source_file': 'nnn2'}}],
  'newSynapses': [{'syn0': {'neuroml2_source_file': None, 'tested': True}},
   {'syn1': {'neuroml2_source_file': 'xx', 'tested': None}}]}}

In [70]:
# Loading json file
loaded_json_2 = load_json('netid.json')
loaded_json_2

{'netid': {'cells': {'cellid1': {'neuroml2_source_file': 'nnn'},
   'cellid2': {'neuroml2_source_file': 'nnn2'}},
  'synapses': {'syn0': {'tested': True},
   'syn1': {'neuroml2_source_file': 'xx'}},
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2]}}

In [77]:
# Loading yaml file
loaded_yaml_2 = load_yaml('netid.json')
loaded_yaml_2

{'netid': {'cells': {'cellid1': {'neuroml2_source_file': 'nnn'},
   'cellid2': {'neuroml2_source_file': 'nnn2'}},
  'synapses': {'syn0': {'tested': True},
   'syn1': {'neuroml2_source_file': 'xx'}},
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2]}}

In [78]:
# Original dict
nml_dict_2 = net.to_dict()
nml_dict_2

{'netid': {'cells': {'cellid1': {'neuroml2_source_file': 'nnn'},
   'cellid2': {'neuroml2_source_file': 'nnn2'}},
  'synapses': {'syn0': {'tested': True},
   'syn1': {'neuroml2_source_file': 'xx'}},
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2]}}

In [79]:
# Comparing loaded xml with original dictionary
check(loaded_xml_2, nml_dict_2)

False


In [80]:
# Comparing loaded xml with loaded json
check(loaded_xml_2, loaded_json_2)

False


In [81]:
# Comparing loaded xml with laoded yaml
check(loaded_xml_2, loaded_yaml_2)

False


In [82]:
nml_doc

NeuroML(id='TestNeuroML', version='NeuroML_v2.3', izhikevich2007Cells=[Izhikevich2007Cell(id='izh2007RS0', C='100pF', v0='-60mV', k='0.7nS_per_mV', vr='-60mV', vt='-40mV', vpeak='35mV', a='0.03per_ms', b='-2nS', c='-50.0mV', d='100pA')], pulseGenerators=[PulseGenerator(id='pulseGen_0', delay='100ms', duration='800ms', amplitude='0.07 nA')], networks=[Network(id='IzNet', populations=[Population(id='IzhPop0', component='izh2007RS0', size=1)], explicitInputs=[ExplicitInput(target='IzhPop0[0]', input='pulseGen_0')])])

In [200]:
datax

{'netid': {'version': 'NeuroMLlite 0.0',
  'seed': None,
  'stable': False,
  'parameters': "{'size': 3, 'name': None}",
  'random_connectivity': 'NewRandomConnectivity(probability=0.01)',
  'ee0': 'str',
  'ee1': "{'a': 2}",
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': '[1, 2]',
  'ee6': None,
  'newCells': [{'cellid1': {'neuroml2_source_file': 'nnn'}},
   {'cellid2': {'neuroml2_source_file': 'nnn2'}}],
  'newSynapses': [{'syn0': {'neuroml2_source_file': None, 'tested': True}},
   {'syn1': {'neuroml2_source_file': 'xx', 'tested': None}}]}}

In [160]:
datay

{'netid': {'cells': {'cellid1': {'neuroml2_source_file': 'nnn'},
   'cellid2': {'neuroml2_source_file': 'nnn2'}},
  'synapses': {'syn0': {'tested': True},
   'syn1': {'neuroml2_source_file': 'xx'}},
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2]}}

In [100]:
netj

NewNetwork(id='IzNet', cells=[], synapses=[], version='NeuroMLlite 0.0', seed=None, stable=None, parameters=None, random_connectivity=None, ee0=None, ee1=None, ee2=None, ee3=None, ee4=None, ee5=None, ee6=None)

In [161]:
nety

NewNetwork(id='netid', cells=[NewCell(id='cellid1', neuroml2_source_file='nnn'), NewCell(id='cellid2', neuroml2_source_file='nnn2')], synapses=[NewSynapse(id='syn0', neuroml2_source_file=None, tested=True), NewSynapse(id='syn1', neuroml2_source_file='xx', tested=None)], version='NeuroMLlite 0.0', seed=None, stable=False, parameters={'size': 3, 'name': None}, random_connectivity=NewRandomConnectivity(probability=0.01), ee0='str', ee1={'a': 2}, ee2=1, ee3=1.1, ee4=True, ee5=[1, 2], ee6=None)

In [196]:
netx

NewNetwork(id='IzNet', cells=[], synapses=[], version='NeuroMLlite 0.0', seed=None, stable=None, parameters=None, random_connectivity=None, ee0=None, ee1=None, ee2=None, ee3=None, ee4=None, ee5=None, ee6=None)

In [177]:
import xml.etree.ElementTree as ET
import attr

def build_xml_element(data, parent=None):
    """
    This recursively builds an XML element structure from a dictionary or a list.

    Args:
        parent: The parent XML element to attach the new element(s) to.
        data: The data to convert into XML elements.

    Returns:
        Parent
    """
    if parent is None:
        parent = ET.Element(data.__class__.__name__)

    attrs = attr.fields(data.__class__)
    for aattr in attrs:
        if isinstance(aattr.default, attr.Factory):
            children = data.__getattribute__(aattr.name)
            if not isinstance(children, (list, tuple)):
                children = [children]

            for child in children:
                child_element = build_xml_element(child)
                parent.append(child_element)
        else:
            attribute_name = aattr.name
            attribute_value = data.__getattribute__(aattr.name)
            parent.set(attribute_name, str(attribute_value))

    return parent


root = build_xml_element(net)

xml_string = ET.tostring(
    root, encoding="utf-8", xml_declaration=False, method="xml"
).decode("utf-8")

parsed_xml = xml.dom.minidom.parseString(xml_string)
pretty_xml = parsed_xml.toprettyxml(indent=" " * 4)
print(pretty_xml)

<?xml version="1.0" ?>
<NewNetwork id="netid" version="NeuroMLlite 0.0" seed="None" stable="False" parameters="{'size': 3, 'name': None}" random_connectivity="NewRandomConnectivity(probability=0.01)" ee0="str" ee1="{'a': 2}" ee2="1" ee3="1.1" ee4="True" ee5="[1, 2]" ee6="None">
    <NewCell id="cellid1" neuroml2_source_file="nnn"/>
    <NewCell id="cellid2" neuroml2_source_file="nnn2"/>
    <NewSynapse id="syn0" neuroml2_source_file="None" tested="True"/>
    <NewSynapse id="syn1" neuroml2_source_file="xx" tested="None"/>
</NewNetwork>



In [174]:
net

NewNetwork(id='netid', cells=[NewCell(id='cellid1', neuroml2_source_file='nnn'), NewCell(id='cellid2', neuroml2_source_file='nnn2')], synapses=[NewSynapse(id='syn0', neuroml2_source_file=None, tested=True), NewSynapse(id='syn1', neuroml2_source_file='xx', tested=None)], version='NeuroMLlite 0.0', seed=None, stable=False, parameters={'size': 3, 'name': None}, random_connectivity=NewRandomConnectivity(probability=0.01), ee0='str', ee1={'a': 2}, ee2=1, ee3=1.1, ee4=True, ee5=[1, 2], ee6=None)

In [181]:
net.to_dict()

{'netid': {'cells': {'cellid1': {'neuroml2_source_file': 'nnn'},
   'cellid2': {'neuroml2_source_file': 'nnn2'}},
  'synapses': {'syn0': {'tested': True},
   'syn1': {'neuroml2_source_file': 'xx'}},
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2]}}

In [185]:
xml_string = nml_doc.to_xml()
print(xml_string)

<?xml version="1.0" ?>
<NeuroML id="TestNeuroML" version="NeuroML_v2.3">
    <Izhikevich2007Cell id="izh2007RS0" C="100pF" v0="-60mV" k="0.7nS_per_mV" vr="-60mV" vt="-40mV" vpeak="35mV" a="0.03per_ms" b="-2nS" c="-50.0mV" d="100pA"/>
    <PulseGenerator id="pulseGen_0" delay="100ms" duration="800ms" amplitude="0.07 nA"/>
    <Network id="IzNet">
        <Population id="IzhPop0" component="izh2007RS0" size="1"/>
        <ExplicitInput target="IzhPop0[0]" input="pulseGen_0"/>
    </Network>
</NeuroML>



In [186]:
net

NewNetwork(id='netid', cells=[NewCell(id='cellid1', neuroml2_source_file='nnn'), NewCell(id='cellid2', neuroml2_source_file='nnn2')], synapses=[NewSynapse(id='syn0', neuroml2_source_file=None, tested=True), NewSynapse(id='syn1', neuroml2_source_file='xx', tested=None)], version='NeuroMLlite 0.0', seed=None, stable=False, parameters={'size': 3, 'name': None}, random_connectivity=NewRandomConnectivity(probability=0.01), ee0='str', ee1={'a': 2}, ee2=1, ee3=1.1, ee4=True, ee5=[1, 2], ee6=None)

In [187]:
net.to_dict()

{'netid': {'cells': {'cellid1': {'neuroml2_source_file': 'nnn'},
   'cellid2': {'neuroml2_source_file': 'nnn2'}},
  'synapses': {'syn0': {'tested': True},
   'syn1': {'neuroml2_source_file': 'xx'}},
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2]}}

In [193]:
import cattr
def to_dict(data) -> Dict[str, Any]:
        """Convert the Base object to a nested dict structure."""
        converter = cattr.Converter()
        d = converter.unstructure(data)

        # If this object has an id attribute, by default lets serialize it within a dict with the id as the key, even if
        # it is a single object.
        if "id" in [f.name for f in attr.fields(data.__class__)] and "id" in d.keys():
            # Drop the id field
            del d["id"]
            return {data.id: d}
        else:
            return d

        
dict_d = to_dict(net)
dict_d

{'netid': {'cells': [{'id': 'cellid1', 'neuroml2_source_file': 'nnn'},
   {'id': 'cellid2', 'neuroml2_source_file': 'nnn2'}],
  'synapses': [{'id': 'syn0', 'neuroml2_source_file': None, 'tested': True},
   {'id': 'syn1', 'neuroml2_source_file': 'xx', 'tested': None}],
  'version': 'NeuroMLlite 0.0',
  'seed': None,
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2],
  'ee6': None}}

In [192]:
net.to_dict()

{'netid': {'cells': {'cellid1': {'neuroml2_source_file': 'nnn'},
   'cellid2': {'neuroml2_source_file': 'nnn2'}},
  'synapses': {'syn0': {'tested': True},
   'syn1': {'neuroml2_source_file': 'xx'}},
  'stable': False,
  'parameters': {'size': 3, 'name': None},
  'random_connectivity': {'probability': 0.01},
  'ee0': 'str',
  'ee1': {'a': 2},
  'ee2': 1,
  'ee3': 1.1,
  'ee4': True,
  'ee5': [1, 2]}}